In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(1, '/home/cem/Documents/imps/src')

import numpy as np
import open3d as o3d
import torch
from matplotlib import cm
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from sklearn.metrics import jaccard_score

import pandas as pd

from imps.data import ScanNetScene, CLASS_NAMES
from imps.sqn.model import SQN
from imps.sqn.data_utils import prepare_input
from imps.ndf.local_model import MiniNDF

SCENE_DIR = '/home/cem/Documents/datasets/ScanNet/scans/scene0000_00'

N_POINTS = int(1.5e5)
RESOLUTION = 125
SIGMAS = np.array([0.5, 0.1, 0.01])

DEVICE = 'cpu'
IGNORED_LABELS = [0]

def get_iou(logits, labels):
    preds = np.argmax(logits, axis=-1)
    ious = []
    
    for c in range(len(CLASS_NAMES)):
        iou = jaccard_score((labels==c).astype(int), (preds==c).astype(int), pos_label=1)
        ious.append(iou)
        
    return np.array(ious)

In [2]:
scene = ScanNetScene(SCENE_DIR)

voxel_grid, surface_points, surface_colors, vicinity_points, vicinity_distances, point_labels = scene.create_if_data(
    RESOLUTION, N_POINTS, SIGMAS
)

class_counts = []
for c in range(len(CLASS_NAMES.keys())):
    class_counts.append(np.sum(point_labels == c))
class_counts = np.array(class_counts)

for ign in IGNORED_LABELS:
    class_counts[ign] = 0
class_weights = class_counts / class_counts.sum()

In [7]:
ndf_model = MiniNDF(is_cuda=False)
ndf_model.load_state_dict(torch.load('../../data/ndf'))
ndf_model.eval();

voxel_input = torch.FloatTensor(voxel_grid).unsqueeze(0)
xyz = torch.FloatTensor(surface_points).unsqueeze(0)
features = torch.FloatTensor(surface_colors).unsqueeze(0)

with torch.no_grad():
    encoding = ndf_model.encoder(voxel_input)
    ndf_features = ndf_model.get_features(xyz, *encoding).permute(0,2,1).detach()

In [8]:
features = torch.cat([ndf_features, features], dim=-1).to(DEVICE)
xyz = torch.FloatTensor(surface_points).unsqueeze(0)

input_points, input_neighbors, input_pools = prepare_input(xyz, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn = SQN(d_feature=797, d_in=16, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn.load_state_dict(torch.load('../../data/sqn-ndf'))
sqn.eval();

In [9]:
with torch.no_grad():
    all_logits = sqn.forward(features, input_points, input_neighbors, input_pools, xyz)
    all_logits = all_logits.squeeze().detach().numpy()

In [10]:
df = pd.DataFrame()
df['class'] = CLASS_NAMES.keys()
df['iou'] = get_iou(all_logits, point_labels)
df['weight'] = class_weights
df['iou_weighted'] = df['iou'] * df['weight']

print("mIOU:", df.iou_weighted.sum())

mIOU: 0.38207630251605224


In [11]:
df

,class,iou,weight,iou_weighted
0,unannotated,0.025148,0.000000,0.000000
1,wall,0.412986,0.268830,0.111023
2,floor,0.499468,0.288269,0.143981
3,cabinet,0.345158,0.147099,0.050772
4,bed,0.360278,0.058880,0.021213
5,chair,0.000000,0.000000,0.000000
6,sofa,0.449182,0.099353,0.044628
7,table,0.000000,0.061126,0.000000
8,door,0.000000,0.012859,0.000000
9,window,0.000000,0.010241,0.000000
